# Trabalhando com *Dataframes* Pandas

**Bibliotecas importadas**

In [11]:
import datetime as dt 
import numpy as np 
import pandas as pd 
import sqlite3

**Carregamento do arquivo**

In [8]:
# Contagem do número de linhas do arquivo 'data/earthquakes.csv'
!wc -l data/earthquakes.csv

9333 data/earthquakes.csv


Usando a biblioteca `pandas` para ler o arquivo 'earthquakes.csv' como um _dataframe_:

In [9]:
df = pd.read_csv('data/earthquakes.csv')

In [10]:
df.head()

,alert,cdi,code,detail,dmin,felt,gap,ids,mag,magType,...,sources,status,time,title,tsunami,type,types,tz,updated,url
0,NaN,NaN,37389218,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.008693,NaN,85.0,",ci37389218,",1.35,ml,...,",ci,",automatic,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475395144,https://earthquake.usgs.gov/earthquakes/eventp...
1,NaN,NaN,37389202,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.020030,NaN,79.0,",ci37389202,",1.29,ml,...,",ci,",automatic,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475253925,https://earthquake.usgs.gov/earthquakes/eventp...
2,NaN,4.4,37389194,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.021370,28.0,21.0,",ci37389194,",3.42,ml,...,",ci,",automatic,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,earthquake,",dyfi,focal-mechanism,geoserve,nearby-cities,o...",-480.0,1539536756176,https://earthquake.usgs.gov/earthquakes/eventp...
3,NaN,NaN,37389186,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.026180,NaN,39.0,",ci37389186,",0.44,ml,...,",ci,",automatic,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,",-480.0,1539475196167,https://earthquake.usgs.gov/earthquakes/eventp...
4,NaN,NaN,73096941,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.077990,NaN,192.0,",nc73096941,",2.16,md,...,",nc,",automatic,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,earthquake,",geoserve,nearby-cities,origin,phase-data,scit...",-480.0,1539477547926,https://earthquake.usgs.gov/earthquakes/eventp...


Leitura de um banco de dados com a biblioteca `sqlite3`.

In [20]:
with sqlite3.connect('./data/quakes.db') as connection:
    pd.read_csv('./data/tsunamis.csv').to_sql(
        'tsunamis', connection, index=False, if_exists='replace'
    )

In [21]:
with sqlite3.connect('./data/quakes.db') as connection:
    query = 'SELECT * FROM tsunamis'
    tsunamis_df = pd.read_sql(query, connection)

tsunamis_df.head()

,alert,type,title,place,magType,mag,time
0,None,earthquake,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...","165km NNW of Flying Fish Cove, Christmas Island",mww,5.0,1539459504090
1,green,earthquake,"M 6.7 - 262km NW of Ozernovskiy, Russia","262km NW of Ozernovskiy, Russia",mww,6.7,1539429023560
2,green,earthquake,"M 5.6 - 128km SE of Kimbe, Papua New Guinea","128km SE of Kimbe, Papua New Guinea",mww,5.6,1539312723620
3,green,earthquake,"M 6.5 - 148km S of Severo-Kuril'sk, Russia","148km S of Severo-Kuril'sk, Russia",mww,6.5,1539213362130
4,green,earthquake,"M 6.2 - 94km SW of Kokopo, Papua New Guinea","94km SW of Kokopo, Papua New Guinea",mww,6.2,1539208835130


Obtendo dados de uma API

In [22]:
# Bibliotecas importadas
import datetime as dt 
import pandas as pd 
import requests

Faremos uma requisição tipo `GET` à API USGS. Nós buscaremos dados relativos a terremotos para os últimos 30 dias.

In [28]:
yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}

response = requests.get(api, params=payload)

print(response.status_code)

200


In [31]:
if response.status_code == 200:
    earthquake_json = response.json()
    print(earthquake_json.keys())

dict_keys(['type', 'metadata', 'features', 'bbox'])


In [32]:
earthquake_json['metadata']

{'generated': 1640984737000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2021-11-30&endtime=2021-12-30',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.12.3',
 'count': 8816}

In [33]:
type(earthquake_json['features'])

list

In [34]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 2.3,
  'place': '60 km ENE of Ouzinkie, Alaska',
  'time': 1640820717890,
  'updated': 1640820961986,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak021gopypu5',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak021gopypu5&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 81,
  'net': 'ak',
  'code': '021gopypu5',
  'ids': ',ak021gopypu5,',
  'sources': ',ak,',
  'types': ',origin,',
  'nst': None,
  'dmin': None,
  'rms': 0.41,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 2.3 - 60 km ENE of Ouzinkie, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-151.5945, 58.1822, 17.7]},
 'id': 'ak021gopypu5'}